In [1]:
import os
os.chdir("..")
print(os.getcwd())

/home/jatin/Projects/customer_churn_prediction


In [2]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    """
    Storing configuration related to the model evaluation.
    """
    root_dir: Path
    test_data_path: Path
    model_path: Path
    target_column: str
    metric_file_name: Path

In [ ]:
from customer_churn_prediction.constants import (
    CONFIG_FILE_PATH,
    SCHEMA_FILE_PATH,
    PARAMS_FILE_PATH
)
from customer_churn_prediction.utils.common import read_yaml, create_directory


class ConfigurationManager:
    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            schema_path=SCHEMA_FILE_PATH,
            params_path=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_path)
        self.schema = read_yaml(schema_path)
        self.params = read_yaml(params_path)

        create_directory([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        Return Model Evaluation config
        """
        config = self.config.model_evaluation
        create_directory([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            target_column = self.schema.TARGET_COLUMN.name,
            metric_file_name = config.metric_file_name
        )
        return model_evaluation_config

In [ ]:
import pandas as pd
import joblib

from sklearn.metrics import ( accuracy_score, auc, fbeta_score, precision_score, recall_score, roc_auc_score )

from customer_churn_prediction.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual,pred)
        recall = recall_score(actual,pred)
        fbeta = fbeta_score(actual, pred)
        roc_auc = roc_auc_score(actual, pred)
        auc_area = auc(actual, pred)
        return accuracy, precision, recall, fbeta, roc_auc, auc_area
    
    def save_result(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        y_pred = model.predict(test_x)
        accuracy, precision, recall, fbeta, roc_auc, auc_area = self.eval_metrics(test_y, y_pred)
        scores = {
            'accuracy':accuracy,
            'precision':precision, 
            'recall': recall, 
            'fbeta':fbeta, 
            'roc_auc':roc_auc, 
            'auc_area': auc_area
        }
        save_json(path = Path(self.config.metric_file_name), data=scores)


In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(model_evaluation_config)
    model_evaluation_config.save_result()
except Exception as e:
    raise e

[2025-11-06 23:01:46,484]:INFO:common.py:Yaml file: config/config.yaml is loaded successfully
[2025-11-06 23:01:46,488]:INFO:common.py:Yaml file: schema.yaml is loaded successfully
[2025-11-06 23:01:46,494]:INFO:common.py:Yaml file: params.yaml is loaded successfully
[2025-11-06 23:01:46,496]:INFO:common.py:Directory created at: artifacts
[2025-11-06 23:01:46,498]:INFO:common.py:Directory created at: artifacts/model_evaluation
[2025-11-06 23:01:46,621]:INFO:common.py:json file saved at: artifacts/model_evaluation/metrics.json
